# ضبط نماذج Open AI بدقة

تعتمد هذه الدفتر على الإرشادات الحالية المقدمة في وثائق [الضبط الدقيق](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) من Open AI.

يعمل الضبط الدقيق على تحسين أداء نماذج الأساس لتطبيقك من خلال إعادة تدريبه ببيانات وسياق إضافي ذي صلة بحالة الاستخدام أو السيناريو المحدد. لاحظ أن تقنيات هندسة المطالبات مثل _التعلم بعدد قليل من الأمثلة_ و _التوليد المعزز بالاسترجاع_ تتيح لك تحسين المطالبة الافتراضية ببيانات ذات صلة لتحسين الجودة. ومع ذلك، فإن هذه الأساليب محدودة بحجم نافذة الرموز القصوى للنموذج الأساسي المستهدف.

مع الضبط الدقيق، نحن فعليًا نعيد تدريب النموذج نفسه بالبيانات المطلوبة (مما يسمح لنا باستخدام عدد أكبر من الأمثلة مما يمكن أن يتسع في نافذة الرموز القصوى) - ونشر نسخة _مخصصة_ من النموذج التي لم تعد بحاجة إلى تقديم أمثلة في وقت الاستدلال. هذا لا يحسن فقط فعالية تصميم المطالبة لدينا (لدينا مرونة أكبر في استخدام نافذة الرموز لأشياء أخرى) ولكنه قد يحسن أيضًا تكاليفنا (عن طريق تقليل عدد الرموز التي نحتاج إلى إرسالها إلى النموذج في وقت الاستدلال).

لضبط الدقيق أربع خطوات:
1. إعداد بيانات التدريب وتحميلها.
1. تشغيل مهمة التدريب للحصول على نموذج مضبوط بدقة.
1. تقييم النموذج المضبوط بدقة والتكرار لتحسين الجودة.
1. نشر النموذج المضبوط بدقة للاستدلال عند الرضا.

لاحظ أن ليس كل نماذج الأساس تدعم الضبط الدقيق - [تحقق من وثائق OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) لأحدث المعلومات. يمكنك أيضًا ضبط نموذج تم ضبطه مسبقًا بدقة. في هذا الدرس، سنستخدم `gpt-35-turbo` كنموذج الأساس المستهدف للضبط الدقيق.

---


### الخطوة 1.1: إعداد مجموعة البيانات الخاصة بك

لنقم ببناء روبوت محادثة يساعدك على فهم الجدول الدوري للعناصر من خلال الإجابة على الأسئلة حول عنصر ما باستخدام ليمريك. في _هذا_ الدرس البسيط، سنقوم فقط بإنشاء مجموعة بيانات لتدريب النموذج مع بعض الأمثلة النموذجية للردود التي توضح التنسيق المتوقع للبيانات. في حالة الاستخدام الواقعي، ستحتاج إلى إنشاء مجموعة بيانات تحتوي على العديد من الأمثلة. قد تتمكن أيضًا من استخدام مجموعة بيانات مفتوحة (لمجال تطبيقك) إذا كانت موجودة، وإعادة تنسيقها لاستخدامها في التخصيص الدقيق.

نظرًا لأننا نركز على `gpt-35-turbo` ونبحث عن استجابة ذات دور واحد (إكمال محادثة) يمكننا إنشاء أمثلة باستخدام [هذا التنسيق المقترح](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) الذي يعكس متطلبات إكمال المحادثة من OpenAI. إذا كنت تتوقع محتوى محادثة متعدد الأدوار، فستستخدم [تنسيق المثال متعدد الأدوار](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) الذي يتضمن معلمة `weight` للإشارة إلى الرسائل التي يجب استخدامها (أو لا) في عملية التخصيص الدقيق.

سنستخدم التنسيق الأبسط ذو الدور الواحد في درسنا هنا. البيانات بتنسيق [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) مع سجل واحد لكل سطر، كل منها ممثل ككائن بصيغة JSON. المقتطف أدناه يعرض سجلين كمثال - انظر [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) لمجموعة الأمثلة الكاملة (10 أمثلة) التي سنستخدمها في درس التخصيص الدقيق. **ملاحظة:** يجب تعريف كل سجل _في سطر واحد_ (ليس مقسماً عبر عدة أسطر كما هو معتاد في ملف JSON منسق)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

في حالة الاستخدام الواقعي ستحتاج إلى مجموعة أمثلة أكبر بكثير للحصول على نتائج جيدة - سيكون المقابل هو بين جودة الردود والوقت/التكاليف للتخصيص الدقيق. نحن نستخدم مجموعة صغيرة حتى نتمكن من إكمال التخصيص بسرعة لتوضيح العملية. انظر [مثال OpenAI Cookbook هذا](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) لدرس تخصيص دقيق أكثر تعقيدًا.


---

### الخطوة 1.2 رفع مجموعة البيانات الخاصة بك

قم برفع البيانات باستخدام واجهة برمجة تطبيقات الملفات [كما هو موضح هنا](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). لاحظ أنه لتشغيل هذا الكود، يجب أن تكون قد قمت بالخطوات التالية أولاً:
 - تثبيت حزمة `openai` الخاصة بلغة بايثون (تأكد من استخدام إصدار >=0.28.0 للحصول على أحدث الميزات)
 - تعيين متغير البيئة `OPENAI_API_KEY` إلى مفتاح API الخاص بـ OpenAI
لمعرفة المزيد، راجع [دليل الإعداد](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) المقدم للدورة.

الآن، قم بتشغيل الكود لإنشاء ملف للرفع من ملف JSONL المحلي الخاص بك.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### الخطوة 2.1: إنشاء مهمة الضبط الدقيق باستخدام SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### الخطوة 2.2: التحقق من حالة الوظيفة

إليك بعض الأشياء التي يمكنك القيام بها باستخدام واجهة برمجة التطبيقات `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - سرد آخر n من وظائف الضبط الدقيق
- `client.fine_tuning.jobs.retrieve(<job_id>)` - الحصول على تفاصيل وظيفة ضبط دقيق محددة
- `client.fine_tuning.jobs.cancel(<job_id>)` - إلغاء وظيفة ضبط دقيق
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - سرد ما يصل إلى n من الأحداث من الوظيفة
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

الخطوة الأولى من العملية هي _التحقق من صحة ملف التدريب_ للتأكد من أن البيانات في التنسيق الصحيح.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### الخطوة 2.3: تتبع الأحداث لمراقبة التقدم


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### الخطوة 2.4: عرض الحالة في لوحة تحكم OpenAI


يمكنك أيضًا عرض الحالة بزيارة موقع OpenAI واستكشاف قسم _الضبط الدقيق_ في المنصة. سيُظهر لك هذا حالة الوظيفة الحالية، ويسمح لك أيضًا بتتبع تاريخ عمليات تنفيذ الوظائف السابقة. في هذه اللقطة، يمكنك أن ترى أن التنفيذ السابق فشل، وأن التشغيل الثاني نجح. للسياق، حدث هذا عندما استخدم التشغيل الأول ملف JSON يحتوي على سجلات بتنسيق غير صحيح - بمجرد إصلاحه، أكمل التشغيل الثاني بنجاح وجعل النموذج متاحًا للاستخدام.

![Fine-tuning job status](../../../../../translated_images/ar/fine-tuned-model-status.563271727bf7bfba.webp)


يمكنك أيضًا عرض رسائل الحالة والقياسات عن طريق التمرير لأسفل أكثر في لوحة المعلومات المرئية كما هو موضح:

| الرسائل | القياسات |
|:---|:---|
| ![Messages](../../../../../translated_images/ar/fine-tuned-messages-panel.4ed0c2da5ea1313b.webp) |  ![Metrics](../../../../../translated_images/ar/fine-tuned-metrics-panel.700d7e4995a65229.webp)|


---

### الخطوة 3.1: استرجاع المعرف واختبار النموذج المحسن في الكود


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### الخطوة 3.2: تحميل واختبار النموذج المُحسّن في الملعب

يمكنك الآن اختبار النموذج المُحسّن بطريقتين. أولاً، يمكنك زيارة الملعب واستخدام قائمة النماذج المنسدلة لاختيار النموذج المُحسّن الجديد من بين الخيارات المدرجة. الخيار الآخر هو استخدام خيار "الملعب" المعروض في لوحة التخصيص (انظر لقطة الشاشة أعلاه) والذي يفتح العرض _المقارن_ التالي الذي يعرض إصدارات النموذج الأساسي والمُحسّن جنبًا إلى جنب لتقييم سريع.

![Fine-tuning job status](../../../../../translated_images/ar/fine-tuned-playground-compare.56e06f0ad8922016.webp)

ببساطة املأ سياق النظام المستخدم في بيانات التدريب الخاصة بك وقدم سؤال الاختبار الخاص بك. ستلاحظ أن كلا الجانبين يتم تحديثهما بالسياق والسؤال المتطابقين. قم بتشغيل المقارنة وسترى الفرق في المخرجات بينهما. _لاحظ كيف يعرض النموذج المُحسّن الرد بالشكل الذي قدمته في أمثلتك بينما يتبع النموذج الأساسي ببساطة موجه النظام_.

![Fine-tuning job status](../../../../../translated_images/ar/fine-tuned-playground-launch.5a26495c983c6350.webp)

ستلاحظ أن المقارنة توفر أيضًا عدد الرموز لكل نموذج، والوقت المستغرق للاستدلال. **هذا المثال المحدد بسيط ويهدف إلى إظهار العملية فقط وليس يعكس مجموعة بيانات أو سيناريو حقيقي**. قد تلاحظ أن كلا العينتين تظهران نفس عدد الرموز (سياق النظام وموجه المستخدم متطابقان) مع استغراق النموذج المُحسّن وقتًا أطول للاستدلال (النموذج المخصص).

في السيناريوهات الواقعية، لن تستخدم مثالًا بسيطًا كهذا، بل ستقوم بالتخصيص مقابل بيانات حقيقية (مثل كتالوج المنتجات لخدمة العملاء) حيث ستكون جودة الاستجابة أكثر وضوحًا. في _ذلك_ السياق، الحصول على جودة استجابة معادلة باستخدام النموذج الأساسي سيتطلب هندسة موجه مخصصة أكثر مما سيزيد من استخدام الرموز وربما الوقت المتعلق بمعالجة الاستدلال. _لتجربة ذلك، اطلع على أمثلة التخصيص في كتاب وصفات OpenAI للبدء._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الموثوق به. للمعلومات الهامة، يُنصح بالاعتماد على الترجمة البشرية المهنية. نحن غير مسؤولين عن أي سوء فهم أو تفسير ناتج عن استخدام هذه الترجمة.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
